### 웹툰 이미지를 다운로드하여 로컬에 저장하기

In [15]:
import requests
import os

req_header_dict = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=552960&no=410&amp;weekday=fri'
}

img_url_list = [
    'https://image-comic.pstatic.net/webtoon/552960/410/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/552960/410/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/552960/410/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_3.jpg'
]

for img_url in img_url_list:
    res = requests.get(img_url, headers=req_header_dict)
    print(res.status_code)
    if res.ok:
        # binary data 가져올때 .content 속성사용
        img_data = res.content
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        file_name = 'data/' + file_name 
        
        # 서버에서 가져온 binary data를 file로 저장하기
        with open(file_name, 'wb') as file:
            print(f'Write to file {file_name} ({len(img_data):,}) bytes')
            file.write(img_data)      

200
Write to file data/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_1.jpg (162,066) bytes
200
Write to file data/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_2.jpg (215,640) bytes
200
Write to file data/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_3.jpg (212,535) bytes


### 웹툰의 특정회차의 모든 image 다운로드 하기
* Attribute Selector를 사용하여 jpg 파일명들을 모두 추출하여 리스트에 저장하기

In [25]:
import requests
from bs4 import BeautifulSoup

main_url = 'https://comic.naver.com/webtoon/detail?titleId=764040&no=54&weekday=fri'
res = requests.get(main_url)

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    img_tags = soup.select("img[src$='.jpg']") 
    print( len(img_tags), type(img_tags) )
    
    img_url_list = []
    for img_tag in img_tags:
        #print(type(img_tag), img_tag)
        img_url = img_tag['src']
        img_url_list.append(img_url)

print(len(img_url_list))
print(img_url_list[:4])

72 <class 'bs4.element.ResultSet'>
72
['https://shared-comic.pstatic.net/thumb/webtoon/764040/thumbnail/thumbnail_IMAG04_8afc9026-2558-49e9-b675-93381d9ef009.jpg', 'https://image-comic.pstatic.net/webtoon/764040/54/20220127142619_b148e6dfbf0f74b0c384744f9e46035c_IMAG01_1.jpg', 'https://image-comic.pstatic.net/webtoon/764040/54/20220127142619_b148e6dfbf0f74b0c384744f9e46035c_IMAG01_2.jpg', 'https://image-comic.pstatic.net/webtoon/764040/54/20220127142619_b148e6dfbf0f74b0c384744f9e46035c_IMAG01_3.jpg']


In [27]:
# img 디렉토리 생성하기
import os

dir_path = 'img'

#img 디렉토리가 없으면
if not os.path.isdir(dir_path):
    os.mkdir(dir_path)

### 디렉토리 생성하는 2가지 함수
* os.mkdir() / osk.makedirs()
 1. os.mkdir() 은 1개 폴더만 생성
 2. os.makedirs() 는 a/b/c 처럼 하위 폴더를 생성

* 리스트를 순회하면서 image 다운로드 하기

In [32]:
for idx, img_url in enumerate(img_url_list, 1):
    #print(f'다운로드 번호 {idx} URL = {img_url}')
    req_header = {'referer':main_url}
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_name = os.path.basename(img_url)
        file_name = 'img/' + file_name
        with open(file_name, 'wb') as file:
            file.write(img_data)

In [34]:
# 파일들이 있는 디렉토리 삭제
import shutil
import os

dir_path = 'img'
#img 디렉토리가 있다면
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)

#### 웹툰의 타이틀과 특정회차 url을 아규먼트 받아서 다운로드 하는 함수 구현하기

In [49]:
def download_image(title, round_url):
    import requests
    from bs4 import BeautifulSoup
    import os
    import shutil
    
    # img 폴더가 있으면 삭제하기
    dir_path = 'img'
    
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        
    #img 디렉토리가 없으면
    if not os.path.isdir(dir_path):
        #img\외모지상주의
        title_path = os.path.join(dir_path, title)
        print(title_path)
        os.makedirs(title_path)
    
    # img url 목록을 알아내기 위한 요청을 보내기
    res = requests.get(round_url)
    if res.ok:
        # 응답으로 받은 html텍스트를 파싱하기 위한 파서(BeautifulSoup)객체생성
        soup = BeautifulSoup(res.text, 'html.parser')
        # img태그 중에서 src 속성의 값이 .jpg로 끝나는 태그들만 선택하기
        img_tags = soup.select("img[src$='.jpg']") 
        print( len(img_tags), type(img_tags) )
        
        # 특정회차 url을 referer 헤더로 설정하기
        req_header = { 'referer':round_url }
        
        for idx, img_tag in enumerate(img_tags,1):
            #img 태그의 src 속성의 값을 추출하기
            print(f'------> 다운로드 번호 {idx}')
            img_url = img_tag['src']
            
            # jpg image 데이터 요청하기
            res_img = requests.get(img_url, headers=req_header)
            if res_img.ok:
                # image binary 데이터 가져오기
                img_data = res_img.content
                
                #img\외모지상주의\thumbnail_IMAG04_be987481-7e58-478d-ae1b-c33228ccc658.jpg
                file_name = title_path + '\\' + os.path.basename(img_url)
                
                with open(file_name, 'wb') as file:
                    print(f'{file_name} ({len(img_data)}) bytes')
                    file.write(img_data)

In [51]:
#download_image('외모지상주의','https://comic.naver.com/webtoon/detail?titleId=641253&no=376&amp;weekday=fri')

### Image File Upload
* https://httpbin.org/post URL로 요청을 보내서 다운로드 받은 img 파일을 업로드 하기
* requests.post() 함수를 사용하고, files 속성에 image data를 지정한다.
* files – (optional) Dictionary of ‘filename’: file-like-objects for multipart encoding upload.

In [56]:
import requests

upload_files_dict = {
    'img1': open('data\\f1.jpg','rb'),
    'img2': open('data\\f2.jpg','rb'),
    'img3': open('data\\f3.jpg','rb')
}

url = 'http://httpbin.org/post'
res = requests.post(url, files=upload_files_dict)
print(res.status_code)
res_data = res.json()
img1_data = res_data['files']['img1']
#print(img1_data)

200
